In [1]:
# Import necessary libraries
import  rdflib
from    rdflib import Graph, Namespace, URIRef, Literal, RDF, RDFS, OWL, Literal, URIRef
from    rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, RDFS
import pyvis
from pyvis.network import Network
from IPython.display import IFrame

# 1. Load the Turtle file into an RDF graph.
# Replace 'your_file.ttl' with the path to your Turtle file.
g = Graph()

rdf     = rdflib.Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs    = rdflib.Namespace("http://www.w3.org/2000/01/rdf-schema#")
gr      = rdflib.Namespace("http://purl.org/goodrelations/v1#")
owl     = rdflib.Namespace("http://www.w3.org/2002/07/owl#")

dbo     = rdflib.Namespace("http://dbpedia.org/ontology/")
dbr     = rdflib.Namespace("http://dbpedia.org/resource/")

g.bind("schema", SDO)
g.bind("dbo",dbo)
g.bind("dbr",dbr)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("gr", gr)
g.bind("owl", owl)
g.bind("xsd", XSD)


SDO = Namespace("https://schema.org/")
g.bind("schema", SDO)

g.parse("top1000ResNutRec.ttl", format="turtle")
print("Turtle file loaded. The graph has {} triples.".format(len(g)))

Turtle file loaded. The graph has 65207 triples.


In [2]:
#4.1
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?name ?inName
WHERE {
    ?recipe a schema:Recipe ;
      schema:hasPart ?ingredient ;
      schema:name ?name .
    ?ingredient schema:name ?inName .
    FILTER (str(?inName) = "mango")
}
"""
qResult = g.query(query)

for row in qResult:
    print(f" {row.name} {row.inName}")

 Mango Salsa mango
 Green Mango Salad With Cilantro Vinaigrette mango
 Fruit Salsa mango


In [3]:
#4.2
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?recipe ?name ?cooktime
WHERE {
  ?recipe a schema:Recipe ;
      schema:cookTime ?cooktime ;
      schema:name ?name;
      schema:recipeCategory "Pie" .
  FILTER( ?cooktime/3600 < 2 )
}
"""
qResult = g.query(query)

print(f"There are {len(qResult)} results")
#for row in qResult:
    #print(f" {row.recipe} {row.name} {row.cooktime}")

There are 30 results


In [4]:
#4.3
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT ?resturant ?name
WHERE {
  ?resturant a schema:Restaurant ;
      schema:name ?name;
      schema:City "New Delhi";
      schema:servesCuisine "Chinese";
      schema:DeliveryMethod "1" .
}
"""
qResult = g.query(query)

for row in qResult:
    print(f" {row.resturant} {row.name} ")

 http://kg-course/foodnutrition/Restaurant/1775 Bengal Sweet Corner 
 http://kg-course/foodnutrition/Restaurant/18089255 Beijing Street 
 http://kg-course/foodnutrition/Restaurant/18128871 Bhappe Di Hatti 
 http://kg-course/foodnutrition/Restaurant/18237363 Bao Cha 
 http://kg-course/foodnutrition/Restaurant/18244555 Asia Central 
 http://kg-course/foodnutrition/Restaurant/18261158 Bemisaal Reloaded 
 http://kg-course/foodnutrition/Restaurant/18268367 Aashirwad Restaurant 
 http://kg-course/foodnutrition/Restaurant/18279452 Beijing Street 
 http://kg-course/foodnutrition/Restaurant/18430600 Bangla Sweets 
 http://kg-course/foodnutrition/Restaurant/18455557 All Day 99 
 http://kg-course/foodnutrition/Restaurant/18458332 AOWLS 
 http://kg-course/foodnutrition/Restaurant/1914 Bhimsain's Bengali Sweet House 
 http://kg-course/foodnutrition/Restaurant/2295 Aroma Rest O Bar 
 http://kg-course/foodnutrition/Restaurant/300275 Anupam Restaurant 
 http://kg-course/foodnutrition/Restaurant/300430

In [5]:
#4.4
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT (AVG(?price) AS ?avgPrice)
WHERE {
  ?resturant a schema:Restaurant ;
      schema:name ?name;
      schema:City "Davenport";
      schema:servesCuisine "Asian"; 
      foodnutrition:costForTwo ?price .
      BIND( (xsd:double(?price)) AS ?price)
}
"""
#find a list of all asian cusines
qResult = g.query(query)

for row in qResult:
    print(f" {row.avgPrice} ")

 0 


In [6]:
#4.5
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?date ?recipe ?name ?img 
WHERE {
  ?recipe a schema:Recipe ;
        schema:name ?name;
        schema:image ?img ;
        schema:keywords "Easy";
        schema:recipeCategory "Dessert" ;
        schema:datePublished ?date .
    ?nutrition a schema:NutritionInformation;
        schema:name ?name;
        schema:calories ?cal .
    FILTER (?date >= "2000-01-01"^^xsd:date)
    FILTER (str(?img) != "character(0)") .
} 
ORDER BY (?cal)
Limit 5
"""
qResult = g.query(query)

for row in qResult:
    print(f" {row.date} {row.recipe} {row.name} {row.img}")

In [12]:
#4.6
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT Distinct ?name
WHERE {
  ?recipe a schema:Recipe ;
      schema:name ?name ;
      schema:recipeCategory "Beverages" .
}
LIMIT 10
"""
qResult = g.query(query)

for row in qResult:
    print(f"  {row.name} ")

  Strawberry Lemonade 
  Cafe Diablo 
  Chocolate Coffee 
  Kahlua 
  Killer Kahlua 
  Cocoa Espresso Cooler 
  Iced Jazzberry Java 
  Best Lemonade 
  Iced Cafe Au Lait 
  Creamy Iced Coffee 


In [ ]:
#4.7
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?name
WHERE {
  ?resturant a schema: ;
      schema:recipeCategory ?name;
}
"""
qResult = g.query(query)

for row in qResult:
    print(f"  {row.name} ")

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '?'  (at char 177), (line:8, col:3)

In [ ]:
#4.8
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dbd:	<http://dbpedia.org/datatype/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?name
WHERE {
  ? a schema: ;
      schema:recipeCategory ?name;
}
"""
qResult = g.query(query)

for row in qResult:
    print(f"  {row.name} ")